In [ ]:
import os
import pandas as pd

monthly_folder = "D:/2020/Monthly Segregated"
monthly_folders = [f for f in os.listdir(monthly_folder) if os.path.isdir(os.path.join(monthly_folder, f))]
column_names = ['Symbol', 'Date', 'Timestamp', 'Open', 'High', 'Low', 'Close', 'Additional_col1', 'Additional_col2']

for month_folder in monthly_folders:
    daily_folder_path = os.path.join(monthly_folder, month_folder)
    daily_files = [f for f in os.listdir(daily_folder_path) if f.endswith('.csv')]
    for file_name in daily_files:
        file_path = os.path.join(daily_folder_path, file_name)
        data = pd.read_csv(file_path, header=None)
        data.columns = column_names
        data.to_csv(file_path, index=False)

In [8]:
import os
import pandas as pd

monthly_segregated_folder = "D:/2020/Monthly Segregated"
total_profit_year = 0
total_loss_year = 0

for month_folder in os.listdir(monthly_segregated_folder):
    daily_folder_path = os.path.join(monthly_segregated_folder, month_folder)
    total_profit = 0
    total_loss = 0
    for csv_file in os.listdir(daily_folder_path):
        data = pd.read_csv(os.path.join(daily_folder_path, csv_file))
        data['open_range'] = data['High'] - data['Low']
        data['first_15_min_high'] = data['High'].rolling(window=15).max()
        data['first_15_min_low'] = data['Low'].rolling(window=15).min()
        data['buy_signal'] = data['High'] > data['first_15_min_high'].shift(1)
        data['sell_signal'] = data['Low'] < data['first_15_min_low'].shift(1)
        in_trade = False  
        entry_price = 0    
        stop_loss_percent = 0.5  
        for index, row in data.iterrows():
            if row['buy_signal'] and not in_trade:
                entry_price = row['Close']
                in_trade = True

            elif row['sell_signal'] and in_trade:
                exit_price = row['Close']
                in_trade = False
                trade_pnl = (exit_price - entry_price) / entry_price * 100  
                if trade_pnl > 0:
                    total_profit += trade_pnl
                else:
                    total_loss += abs(trade_pnl)

    print(f'Total Profit for {month_folder}: {total_profit}')
    print(f'Total Loss for {month_folder}: {total_loss}')
    print()

    total_profit_year += total_profit
    total_loss_year += total_loss

print(f'Total Profit for the Year: {total_profit_year}')
print(f'Total Loss for the Year: {total_loss_year}\n')
print()

Total Profit for April: 173.49994444210733
Total Loss for April: 139.28308897741454

Total Profit for August: 120.05787075761994
Total Loss for August: 120.42677051506871

Total Profit for December: 108.93960071525545
Total Loss for December: 140.36607111468066

Total Profit for February: 113.06192018964445
Total Loss for February: 123.96949093205511

Total Profit for January: 108.90528475950924
Total Loss for January: 126.86991770115976

Total Profit for July: 96.8621322840097
Total Loss for July: 132.58243671337843

Total Profit for June: 170.92703179825872
Total Loss for June: 168.6171652118846

Total Profit for March: 349.7790346339178
Total Loss for March: 353.4170886163639

Total Profit for May: 145.43318608886912
Total Loss for May: 135.8046439159322

Total Profit for November: 105.99399910837607
Total Loss for November: 188.25058020526572

Total Profit for October: 116.1557917202169
Total Loss for October: 130.09594078372945

Total Profit for September: 129.0665970143344
Total 

In [11]:
total_trades = len(data[data['buy_signal'] | data['sell_signal']])
winning_trades = len(data[data['buy_signal'] & (data['Close'] > data['Close'].shift(1))]) + len(data[data['sell_signal'] & (data['Close'] < data['Close'].shift(1))])
losing_trades = total_trades - winning_trades
win_rate = winning_trades / total_trades if total_trades > 0 else 0
average_profit_per_trade = total_profit / winning_trades if winning_trades > 0 else 0
average_loss_per_trade = total_loss / losing_trades if losing_trades > 0 else 0

print(f'Total Trades: {total_trades}')
print(f'Winning Trades: {winning_trades}')
print(f'Losing Trades: {losing_trades}')
print(f'Win Rate: {win_rate * 100:.2f}%')
print(f'Average Profit per Trade: {average_profit_per_trade:.2f} Rupees')
print(f'Average Loss per Trade: {average_loss_per_trade:.2f} Rupees')


Total Trades: 1829
Winning Trades: 1427
Losing Trades: 402
Win Rate: 78.02%
Average Profit per Trade: 0.09 Rupees
Average Loss per Trade: 0.34 Rupees
